# Data Cleaning
Raw data is imported from square.com and cleaned to use for future analyses. Some relevant features impacting the sales, such as weather and social media, are added.

- [Addition of columns](#Addition-of-columns)
- [Addition of weather data](#Addition-of-Weather-Data)
- [Addition of Untappd data](#Addition-of-Untappd-Data)
- [Addition of Instagram data](#Addition-of-Instagram-Data)
- [Export to csv](#Export-to-csv)

In [ ]:
import pandas as pd
import datetime as dt
import numpy as np

In [ ]:
#Square data is imported from the years in question
sales_2019 = pd.read_csv('insert_path')
sales_2020 = pd.read_csv('insert_path')
sales_2021 = pd.read_csv('insert_path')
sales_2022 = pd.read_csv('insert_path')
sales_2023 = pd.read_csv('insert_path')

df_list = [sales_2019, sales_2020, sales_2021, sales_2022, sales_2023]
sales_data = pd.concat(df_list, join='inner', axis='rows')

If the Item values are not consistent, it may be prudent to go through the values and map them to a consistent naming system. In the case of the brewery sales data, I mapped them to groups of Draft, Can, Keg, Flight, Merchandise, Food, or Other for **'product_type'** or to names of specific beers for **'beer_style'** (i.e. IPA 12oz., IPA Can, IPA - Keg and IPA 16oz. all map to IPA). 

In [ ]:
import ast
def read_text_file(file_path):
    with open(file_path, "r") as file:
        content = file.read()
    return content
file_path_style = "insert_path.txt"
file_content_style = read_text_file(file_path_style)
style_dict = ast.literal_eval(file_content_style)

file_path_product = "insert_path.txt"
file_content_product = read_text_file(file_path_product)
product_dict = ast.literal_eval(file_content_product)

sales_data['beer_style'] = sales_data['Item'].map(style_dict)
sales_data['product_type'] = sales_data['Item'].map(product_dict)

In [ ]:
monetary_columns = ['Gross Sales', 'Discounts', 'Net Sales', 'Tax']
for col in monetary_columns:
    sales_data[col] = sales_data[col].str.replace('[$,]', '', regex=True)
    sales_data[col] = sales_data[col].astype('float')
    sales_data[col] = np.round(sales_data[col], decimals = 2)

In [ ]:
#changing applicable columns to datetime
sales_data['Date'] = pd.to_datetime(sales_data['Date'])
sales_data['Time'] = pd.to_datetime(sales_data['Time'])

#adding new columns based off datetime
sales_data['year'] = sales_data['Date'].dt.year
sales_data['month'] = sales_data['Date'].dt.month
sales_data['month_name'] = sales_data['Date'].dt.month_name()
sales_data['day'] = sales_data['Date'].dt.day
sales_data['weekday'] = sales_data['Date'].dt.day_name()
sales_data['hour'] = sales_data['Time'].dt.hour

In [ ]:
sales_data['Customer Name'] = sales_data['Customer Name'].str.replace(',', '', regex=True)
sales_data['Customer Name Init'] = sales_data['Customer Name'].str.replace('[^A-Z]', '', regex=True) #changing customer names to initials for privacy
sales_data['customer_id_no'] = pd.factorize(sales_data['Customer Name'])[0]      

sales_data = sales_data.drop(['Customer Name'], axis='columns')

A unit price column is needed as square only provides the gross sales per item and the quanity.

In [ ]:
sales_data['unit_price'] = sales_data['Gross Sales'] / sales_data['Count'] # adding a price per unit of product column

In [ ]:
sales_data = sales_data.drop(['Payment ID', 'Time', 'Customer ID', 'Customer Reference ID', 
              'Price Point Name', 'Discounts', 'Net Sales', 'Tax', 
              'Category', 'Commission', 'Employee',
              'Unit', 'Dining Option', 'Details', 'Qty', 'SKU', 
              'Modifiers Applied','Location', 
              'Itemization Type', 'Fulfillment Note', 'Notes', 
              'Device Name', 'Event Type', 'Time Zone'], axis='columns')

#names simplified
sales_data = sales_data.rename(columns={"Date":"date",
                        "Item":"item",
                        "Gross Sales":"gross_sales",
                        "Transaction ID":"transaction_id",
                        "Customer Name Init":"customer_init", 
                        "Count":"count"})

sales_data = sales_data.sort_values(by="date", ascending=True, ignore_index=True)

Columns were reordered and sorted chronologically for easier viewing.

In [ ]:
sales_data = sales_data[['date', 'weekday', 'month_name', 'month', 'day', 'year', 'hour', 'item', 'beer_style', 'product_type', 'unit_price', 'count', 'gross_sales', 'transaction_id',
       'customer_init', 'customer_id_no']].sort_values(['date','hour'], ascending=True)

In [ ]:
sales_data.head(20)

## Addition of columns

Square data has lines for each individual item type purchased. The transaction id numbers are long and complex. For the purposes of tracking more easily, factorize can be used to have simpler ID numbers for transactions.

In [ ]:
sales_data['transaction_no'] = pd.factorize(sales_data['transaction_id'])[0]

A customer can be tracked throughout time on their customer ID number. It may be useful to add the frequency of the visitor as a new column so percentage of returning customers on a given day can be determined.

In [ ]:
sales_data = sales_data.sort_values(['customer_id_no', 'date'])
sales_data['visit_freq'] = sales_data.groupby('customer_id_no')['transaction_no'].transform('nunique')

Number of customers on a given day

In [ ]:
sales_data = sales_data.sort_values(['date', 'customer_id_no'])
sales_data['daily_no_customers'] = sales_data.groupby('date')['transaction_no'].transform('nunique')

Number of styles available every day can be impactful in the overall sales. For the purposes of my study, I assumed that every available style was sold in a given day. If this data is available in another data source it is recommended to merge that data on date.

In [ ]:
sales_data = sales_data.sort_values(['date', 'beer_style'])
sales_data['no_styles_sold'] = sales_data.groupby('date')['beer_style'].transform('nunique')

Any customer initial that is blank is a cash sales.

In [ ]:
sales_data['customer_init'].fillna('Cash',inplace=True)

## Addition of Weather Data

A csv file can be retrieved from National Centers for Environmental Information for closest station to the business in question. Be sure to match the date range to that of the sales data.

In [ ]:
weather_data = pd.read_csv('data/weather/weather_data.csv')
weather_data['DATE'] = pd.to_datetime(weather_data['DATE'])
weather = weather_data[['DATE', 'PRCP','SNOW','TMAX','TMIN','PSUN']]
weather = weather.drop(columns=['PSUN'], axis=1)
weather = weather.rename(columns={'DATE':'date',
                                  'PRCP':'precipitation',
                                  'SNOW':'snowfall',
                                  'TMAX':'max_temp',
                                  'TMIN':'min_temp'})

weather.info()

In [ ]:
# add weather to overall dataframe
sales_data = pd.merge(sales_data, weather, on='date', how='left')

## Addition of Untappd Data

[Untappd](https://untappd.com/) is a social media site used for checking in and rating different beers. This data can provide some insight into how well a beer is received by the clientele. Given the typical user of Untappd is an avid beer drinker, this data is somewhat biased. Data can be scraped or downloaded as a csv through a paid account.

In [ ]:
style_data = pd.read_csv('data/beer_style/beer_style_data.csv')
style_data = style_data[['beer_style', 'primary_beer_type','beer_abv',
                         'beer_ibu','global_rating','rating_count','date_added']]
style_data.head()

In [ ]:
sales_data = pd.merge(sales_data, style_data, on='beer_style', how='left')
sales_data.head()

In [ ]:
#fill null values in the event a particular beer has not been added to Untappd
sales_data['rating_count'].fillna(0,inplace=True)
sales_data['beer_abv'].fillna(0,inplace=True)
sales_data['beer_ibu'].fillna(0,inplace=True)
sales_data['global_rating'].fillna(0,inplace=True)
sales_data['primary_beer_type'].fillna('None',inplace=True)

In [ ]:
#revise types as needed
sales_data['rating_count'] = sales_data['rating_count'].astype('int')
sales_data['date_added'] = pd.to_datetime(sales_data['date_added'])

## Addition of Instagram Data

Social media presence can be critical to a small business' success. I manually compiled the data from the breweries instagram noting the date of each post, which beer was promoted, the type of post (promoting an event, a beer, or general promotion), and the days since the previous post. It may also be important to bring in data from other social media sites.

In [ ]:
insta_data = pd.read_csv('data/instagram_posts.csv')
insta_data['date'] = pd.to_datetime(insta_data['date'])
insta_data = insta_data.rename(columns={'type':'insta_post_type'})

insta_data.head()

In [ ]:
#add whether or not there was a post on a given day
day_posted = list(insta_data['date'])
sales_data['insta_post'] = sales_data['date'].apply(lambda x: 1 if x in day_posted else 0)

In [ ]:
insta_data = insta_data[['date','insta_post_type']]

sales_data = pd.merge(sales_data, insta_data, on='date', how='left')
sales_data['insta_post_type'].fillna('No Post',inplace=True)

In [ ]:
sales_data.columns

## Export to csv

In [ ]:
sales_data.to_csv('data/square/export/sales_data_clean.csv')

# Column information

- **date**: date of a given purchase
- **time**: time of day of a given purchase
- **item**: the given name of a product by the brewery
- **gross_sales**: total sales of a given item
- **transaction_id**: a unique ID for a given transaction of a single item type, there can be multiple lines per customer per day under the same ID
- **customer_name**: customer name if credit card used
- **count**: count of a single item type purchased
- **year**: year pulled from date column
- **month**: month pulled from date column
- **month_name**: name of month pulled from date column
- **day**: day pulled from date column
- **weekday**: year pulled from date column
- **hour**: hour of the day of the transaction
- **customer_init**: initials of the customer pulled from the customer_name column
- **unit_price**: price of a single unit of the item
- **beer_style**: brewer given name of the beer, says None if not a beer purchase
- **product_type**: type of product sold: Draft, Can, Keg, Growler, Flight, Other
- **customer_id_no**: number assigned to a given customer based off the customer name (factorize)
- **transaction_no**: number assigned to a single transaction based off the transaction id as it spans multiple lines (factorize)
- **visit_freq**: the number of times the customer in the given row has visited the brewery (transform by nunique)
- **daily_no_customers**: the count of customers on a given day (groupby date and transaction number and transform by nunique)
- **no_styles_sold**: the number of different styles sold on a given day, it is assumed this is the same as the number available
- **precipitation**: rainfall in inches
- **snowfall**: snowfall in inches
- **max_temp**: maximum temperature in Fahrenheit
- **min_temp**: minimum temperature in Fahrenheit
- **sun_percent**: percentage of sunlight
- **primary_beer_type**: the style of the beer as classified by Untappd
- **beer_abv**: beer alcohol by volume
- **beer_ibu**: beer international bitterness unit
- **global_rating**: the average rating across all users who checked in a given beer style on Untappd
- **rating_count**: the count of all users who checked in a given beer style on Untappd
- **date_added**: date a beer was added to Untappd by the brewer
- **insta_post**: binary indicating whether or not there was an instagram post on a given day
- **insta_post_type**: No Post, Event, Beer, None
